---
title: "Presidential Approval Ratings"
format: 
  html:
    echo: false
    embed-resources: true
include-in-header:
      - text: |
            <script src="https://cdnjs.cloudflare.com/ajax/libs/react/18.2.0/umd/react.production.min.js"></script>
            <script src="https://cdnjs.cloudflare.com/ajax/libs/react-dom/18.2.0/umd/react-dom.production.min.js"></script>
            <script src="https://cdnjs.cloudflare.com/ajax/libs/prop-types/15.8.1/prop-types.min.js"></script>
            <script src="https://cdnjs.cloudflare.com/ajax/libs/recharts/2.10.3/Recharts.min.js"></script>
            <script src="https://cdnjs.cloudflare.com/ajax/libs/html2canvas/1.4.1/html2canvas.min.js"></script>
---

In [ ]:
#| output: asis
import polars as pl
import json

# Read the CSV file
df = pl.read_csv('presidential_approval.csv')

# Create a list to store the processed data
presidents_data = []

# Get unique presidents
unique_presidents = df.get_column('president').unique().to_list()

for president in unique_presidents:
    president_df = df.filter(pl.col('president') == president)
    years = president_df.get_column('years')[0]
    
    data_points = []
    for row in president_df.iter_rows(named=True):
        data_points.append({
            'month': int(row['year_in_term'] * 12),
            'approval': row['approval'],
            'disapproval': row['disapproval']
        })
    
    president_obj = {
        'name': president,
        'years': years,
        'data': data_points
    }
    
    presidents_data.append(president_obj)

print(f"<script>const presidentsData = {json.dumps(presidents_data)};</script>")

```{=html}
<div class="visualization-container" style="position: relative; max-width: 1200px; margin: 0 auto; background-color: white;">
    <!-- Content to be exported -->
    <div class="export-content" style="background-color: white;">
        <div style="text-align: center; padding: 20px 0 30px 0;">
            <h1 style="margin: 0; font-size: 2em; color: #333;">Presidential Approval Ratings</h1>
            <p style="margin: 10px 0 0 0; font-size: 1.1em; color: #666;">Tracking Presidential Performance Through the Years</p>
        </div>
        <div id="presidential-approval"></div>
    </div>
    
    <!-- Button container -->
    <div style="text-align: right; padding: 20px;">
        <button id="export-button" style="padding: 10px 20px; background-color: #4CAF50; color: white; border: none; border-radius: 4px; cursor: pointer;">
            Export as PNG
        </button>
    </div>
</div>

<script>
const { LineChart, Line, XAxis, YAxis, CartesianGrid, ResponsiveContainer, Tooltip, Legend } = Recharts;

const CustomTooltip = ({ active, payload, label }) => {
  if (active && payload && payload.length) {
    return React.createElement('div', {
      style: {
        backgroundColor: 'white',
        padding: '8px',
        border: '1px solid #ccc',
        borderRadius: '4px'
      }
    }, [
      React.createElement('p', { key: 'year', style: { fontWeight: 'bold' } }, `Year ${label/12}`),
      React.createElement('p', { key: 'approval' }, `Approval: ${payload[0].value}%`),
      React.createElement('p', { key: 'disapproval', style: { color: '#4CAF50' } }, `Disapproval: ${payload[1].value}%`)
    ]);
  }
  return null;
};

const App = () => {
  return React.createElement('div', {
    style: {
      display: 'grid',
      gridTemplateColumns: 'repeat(3, 1fr)',
      gap: '1.5rem',
      padding: '1rem',
      width: '100%',
      backgroundColor: 'white'
    }
  }, presidentsData.map(president => 
    React.createElement('div', {
      key: president.name,
      style: {
        border: '1px solid #ccc',
        borderRadius: '8px',
        padding: '1rem',
        backgroundColor: 'white'
      }
    }, [
      React.createElement('div', {
        key: 'header',
        style: { 
          fontSize: '1.125rem',
          fontWeight: '600',
          marginBottom: '0.25rem'
        }
      }, [
        president.name,
        React.createElement('span', {
          key: 'years',
          style: {
            color: '#666',
            fontSize: '0.875rem',
            marginLeft: '0.5rem'
          }
        }, president.years)
      ]),
      React.createElement('div', {
        key: 'chart',
        style: { height: '240px' }
      }, React.createElement(ResponsiveContainer, { width: '100%', height: '100%' },
        React.createElement(LineChart, {
          data: president.data,
          margin: { top: 5, right: 30, bottom: 10, left: 5 }
        }, [
          React.createElement(CartesianGrid, { key: 'grid', strokeDasharray: '3 3' }),
          React.createElement(XAxis, {
            key: 'xaxis',
            dataKey: 'month',
            type: 'number',
            domain: [0, 48],
            ticks: [0, 12, 24, 36, 48],
            label: { value: 'Years', position: 'bottom' }
          }),
          React.createElement(YAxis, {
            key: 'yaxis',
            domain: [20, 80],
            ticks: [20, 50, 80]
          }),
          React.createElement(Tooltip, {
            key: 'tooltip',
            content: CustomTooltip
          }),
          React.createElement(Legend, {
            key: 'legend',
            verticalAlign: 'bottom',
            align: 'center',
            wrapperStyle: { paddingTop: '30px' },
            iconType: 'circle'
          }),
          React.createElement(Line, {
            key: 'approval',
            type: 'monotone',
            dataKey: 'approval',
            name: 'Approval',
            stroke: '#000000',
            dot: false,
            strokeWidth: 2,
            activeDot: { r: 4 }
          }),
          React.createElement(Line, {
            key: 'disapproval',
            type: 'monotone',
            dataKey: 'disapproval',
            name: 'Disapproval',
            stroke: '#4CAF50',
            dot: false,
            strokeWidth: 2,
            activeDot: { r: 4 }
          })
        ])
      ))
    ])
  ));
};

// Render the app
const container = document.getElementById('presidential-approval');
const root = ReactDOM.createRoot(container);
root.render(React.createElement(App));

// Add export functionality
document.getElementById('export-button').addEventListener('click', async () => {
    try {
        await new Promise(resolve => setTimeout(resolve, 500));
        const element = document.querySelector('.visualization-container');
        if (!element) {
            throw new Error('Could not find the visualization container');
        }
        
        const canvas = await html2canvas(element, {
            backgroundColor: 'white',
            scale: 2,
            logging: true,
            width: element.offsetWidth,
            height: element.offsetHeight,
            useCORS: true
        });
        
        const link = document.createElement('a');
        link.download = 'presidential_approval_ratings.png';
        link.href = canvas.toDataURL('image/png');
        link.click();
    } catch (error) {
        console.error('Error during export:', error);
        alert('There was an error exporting the chart. Please check the console for details.');
    }
});
</script>

<style>
#presidential-approval {
  width: 100%;
  margin: 0 auto;
}

#export-button:hover {
  background-color: #45a049;
}

.visualization-container {
  background-color: white;
}
</style>
```